In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent.parent)) ## ../../covid19datascience* <- set parent level here

In [ ]:
import pandas as pd
from os.path import join
from bulletin import default_output, default_input, root
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.services.metabase import Metabase
from bulletin.utils.static import Municipios
from bulletin.utils.normalize import normalize_hash, normalize_text, date_hash

In [ ]:
def check_duplicates(db):
    assert not db is None

    db['duplicated'] = False
    for col in [ col for col in db.columns if ('hash' in col) or (col in ['cpf','cns']) ]:
        print(f"duplicated in {col}: {len(db.loc[(db[col].notna())&(db.duplicated(col,keep=False))])}")
        db.loc[(db[col].notna())&(db.duplicated(col,keep='last')),'duplicated'] = True

In [ ]:
database_dir = join(root, 'database', 'notifica')

In [ ]:
mn = Municipios()
mn = mn[['ibge7', 'ibge', 'municipio']]
mn['ibge7'] = mn['ibge7'].astype('str').apply(lambda x: x.zfill(2))
mn['municipio_normalizado'] = mn['municipio'].apply(lambda x: normalize_text(x))
mn = mn.rename(columns={'ibge':'ibge_residencia'})

In [ ]:
cc = CasosConfirmados()
cc.load('cc_05_12_2021', compress=True)

In [ ]:
# cc.df.loc[cc.df['data_nascimento'].notna(),'hash_nasc'] = ( cc.df.loc[cc.df['data_nascimento'].notna(),'paciente'].apply(normalize_hash) + cc.df.loc[cc.df['data_nascimento'].notna(),'data_nascimento'].apply(date_hash) )
# cc.df.loc[cc.df['nome_mae'].notna(),'hash_mae'] = ( cc.df.loc[cc.df['nome_mae'].notna(),'paciente'].apply(normalize_hash) + cc.df.loc[cc.df['nome_mae'].notna(),'nome_mae'].apply(normalize_hash) )

In [ ]:
nf = Notifica()
#nf.databases()
nf.load('notifica', compress=False)
nf.df.loc[(nf.df['classificacao_final'] == 2)]
nf.check_duplicates()

In [ ]:
caso_curitiba = nf.df.loc[(nf.df['ibge_residencia'] == 410690)]

In [ ]:
caso_curitiba = caso_curitiba.loc[caso_curitiba['data_confirmacao'].isin(['2021-07-24', '2021-07-23'])].copy()
print('Casos notificados por Curitiba dia 23 e dia 24:')
caso_curitiba.shape[0]

In [ ]:
del nf

In [ ]:
less = lambda x: str(x-1)
more = lambda x: str(x+1)

In [ ]:
# caso_curitiba['hash_resid_less'] = caso_curitiba['paciente'].apply(normalize_hash) + caso_curitiba['idade'].apply(less) + caso_curitiba['ibge_residencia'].astype(str)
# caso_curitiba['hash_resid_more'] = caso_curitiba['paciente'].apply(normalize_hash) + caso_curitiba['idade'].apply(more) + caso_curitiba['ibge_residencia'].astype(str)
# caso_curitiba['hash_atend_less'] = caso_curitiba['paciente'].apply(normalize_hash) + caso_curitiba['idade'].apply(less) + caso_curitiba['ibge_unidade_notifica'].astype(str)
# caso_curitiba['hash_atend_more'] = caso_curitiba['paciente'].apply(normalize_hash) + caso_curitiba['idade'].apply(more) + caso_curitiba['ibge_unidade_notifica'].astype(str)

In [ ]:
print('Casos notificados por Curitiba dia 23 e dia 24 que entraram no dia 26:')
curitiba_d26= cc.df.loc[(cc.df['data_comunicacao'] == '2021-07-26') & (cc.df['ibge_residencia'] == 410690)]
curitiba_d26.shape[0]

In [ ]:
caso_ativo = caso_curitiba.loc[(caso_curitiba['status_notificacao'].isin([1, 2]))]
print('Fichas notificadas abertas ou encerradas:')
caso_ativo.shape[0]

In [ ]:
curitiba_entraram_ativo = caso_ativo.loc[(
    (caso_ativo['id'].isin(curitiba_d26['id_notifica']))
)]
curitiba_entraram_ativo.shape[0]

In [ ]:
caso_ativo_encontrado = cc.df.loc[(
    (cc.df['id_notifica'].notna() & cc.df['id_notifica'].isin(caso_ativo['id'])) |
    (cc.df['hash_mae'].notna() & cc.df['hash_mae'].isin(caso_ativo['hash_mae'])) |
    (cc.df['hash_nasc'].notna() & cc.df['hash_nasc'].isin(caso_ativo['hash_nasc'])) 
)]
print('Casos ativos encontrado no pickle:')
caso_ativo_encontrado.shape[0]
#caso_ativo_encontrado.drop_duplicates(['id_notifica', 'hash_mae', 'hash_nasc', 'hash_resid', 'hash_resid_less', 'hash_resid_more', 'hash_atend', 'hash_atend_less', 'hash_atend_more','hash_diag'])

In [ ]:
group = caso_ativo_encontrado.groupby('ibge_residencia')[['ibge_residencia']].count().rename(columns={'ibge_residencia':'qtde'}).sort_values('qtde').reset_index()
print('Casos ativos encontrado no pickle através do id do notifica com residência em Curitiba:')
group.loc[group['ibge_residencia'] == 410690, 'qtde'].values[0]

In [ ]:
#print('*CANDITADOS A MUDANÇA DE MUNICIPIO PARA CURITIBA*')
print('Casos ativos encontrado no pickle através do id do notifica com residência outros municipio:')
caso_ativo_encontrado_curitiba = caso_ativo_encontrado.loc[caso_ativo_encontrado['ibge_residencia'] == 410690]
caso_ativo_encontrado_outros = caso_ativo_encontrado.loc[caso_ativo_encontrado['ibge_residencia'] != 410690].copy()
#caso_ativo_encontrado_outros.to_csv(join(default_output,'CANDITADOS_ATIVOS_ALTERACAO_DE_MUNICIPIO.csv'), encoding='utf-8-sig', sep=';')
caso_ativo_encontrado_outros.shape[0]

In [ ]:
caso_ativo_encontrado_outros['ibge_residencia_antigo'] = caso_ativo_encontrado_outros['ibge_residencia']
caso_ativo_encontrado_outros.loc[caso_ativo_encontrado_outros['ibge_residencia'] != 410690, 'ibge_residencia'] = 410690
caso_ativo_encontrado_outros['hash_resid'] = caso_ativo_encontrado_outros['paciente'].apply(normalize_hash) + caso_ativo_encontrado_outros['ibge_residencia'].astype(str)
caso_ativo_encontrado_outros['hash_resid_less'] = caso_ativo_encontrado_outros['paciente'].apply(normalize_hash) + caso_ativo_encontrado_outros['idade'].apply(less) + caso_ativo_encontrado_outros['ibge_residencia'].astype(str)
caso_ativo_encontrado_outros['hash_resid_more'] = caso_ativo_encontrado_outros['paciente'].apply(normalize_hash) + caso_ativo_encontrado_outros['idade'].apply(more) + caso_ativo_encontrado_outros['ibge_residencia'].astype(str)

In [ ]:
canditados_ativos_alteracao_de_municipio = caso_ativo_encontrado_outros.loc[~(
    (caso_ativo_encontrado_outros['hash_resid'].isin(cc.df['hash_resid']))
)]
print('*CANDITADOS A MUDANÇA DE MUNICIPIO PARA CURITIBA*')
canditados_ativos_alteracao_de_municipio.shape[0]

In [ ]:
canditados_ativos_alteracao_de_municipio = canditados_ativos_alteracao_de_municipio.merge(how = 'left', on = 'ibge_residencia', right=mn)
canditados_ativos_alteracao_de_municipio = canditados_ativos_alteracao_de_municipio.merge(how = 'left', on = 'ibge_residencia_antigo', right=mn[['ibge_residencia', 'municipio']].rename(columns={'ibge_residencia':'ibge_residencia_antigo', 'municipio':'municipio_antigo'}))
canditados_ativos_alteracao_de_municipio.loc[canditados_ativos_alteracao_de_municipio['evolucao']==2, 'obito'] = 'SIM'
#canditados_ativos_alteracao_de_municipio[['municipio_antigo','municipio','obito', 'sexo', 'idade']].to_csv(join(default_output,'canditados_ativos_alteracao_de_municipio_relatorio.csv'), encoding='utf-8-sig', sep=';')
#canditados_ativos_alteracao_de_municipio[['identificacao','ibge7','municipio_normalizado']].rename(columns={'identificacao':'Identificacao', 'ibge7':'ibge', 'municipio_normalizado':'mun resid'}).to_csv(join(default_output,'canditados_ativos_alteracao_de_municipio.csv'), encoding='utf-8-sig', sep=';')
#canditados_ativos_alteracao_de_municipio[].to_csv(join(default_output,'canditados_ativos_alteracao_de_municipio.csv'), encoding='utf-8-sig', sep=';')

In [ ]:
caso_ativo_nao_encontrado = caso_ativo.loc[~(
    (caso_ativo['id'].notna() & caso_ativo['id'].isin(cc.df['id_notifica'])) |
    (caso_ativo['hash_mae'].notna() & caso_ativo['hash_mae'].isin(cc.df['hash_mae'])) |
    (caso_ativo['hash_nasc'].notna() & caso_ativo['hash_nasc'].isin(cc.df['hash_nasc'])) 
)].copy()
print('*VERIFICAR PORQUE NAO ENTROU*')
print('Casos ativos não encontrado no pickle:')
# caso_ativo_nao_encontrado.to_csv(join(database_dir,'caso_ativo_nao_encontrado.csv'), encoding='utf-8-sig', sep=';', index=False)
# caso_ativo_nao_encontrado.to_pickle(join(database_dir,'caso_ativo_nao_encontrado.pkl'),'bz2')
caso_ativo_nao_encontrado.shape[0]

In [ ]:
caso_inativo = caso_curitiba.loc[(caso_curitiba['status_notificacao'].isin([3]))]
print('Fichas notificadas inativas:')
caso_inativo.shape[0]

In [ ]:
curitiba_entraram_inativo = caso_inativo.loc[(
    (caso_inativo['id'].isin(curitiba_d26['id_notifica']))
)]
curitiba_entraram_inativo.shape[0]

In [ ]:
caso_inativo_encontrado = cc.df.loc[(
    (cc.df['id_notifica'].notna() & cc.df['id_notifica'].isin(caso_inativo['id'])) |
    (cc.df['hash_mae'].notna() & cc.df['hash_mae'].isin(caso_inativo['hash_mae'])) |
    (cc.df['hash_nasc'].notna() & cc.df['hash_nasc'].isin(caso_inativo['hash_nasc'])) 
)]
print('Casos inativos encontrado no pickle através do id do notifica:')
caso_inativo_encontrado.shape[0]

In [ ]:
group = caso_inativo_encontrado.groupby('ibge_residencia')[['ibge_residencia']].count().rename(columns={'ibge_residencia':'qtde'}).sort_values('qtde').reset_index()
print('Casos inativos encontrado no pickle através do id do notifica com residência em Curitiba:')
group.loc[group['ibge_residencia'] == 410690, 'qtde'].values[0]

In [ ]:
#print('*CANDITADOS INATIVOS A MUDANÇA DE MUNICIPIO PARA CURITIBA*')
print('Casos inativos encontrado no pickle através do id do notifica com residência em outros municipios:')
caso_inativo_encontrado_curitiba = caso_inativo_encontrado.loc[caso_inativo_encontrado['ibge_residencia'] == 410690]
caso_inativo_encontrado_outros = caso_inativo_encontrado.loc[caso_inativo_encontrado['ibge_residencia'] != 410690].copy()
#caso_inativo_encontrado_outros.to_csv(join(default_output,'CANDITADOS_INATIVOS_ALTERACAO_DE_MUNICIPIO_.csv'), encoding='utf-8-sig', sep=';')
caso_inativo_encontrado_outros.shape[0]

In [ ]:
caso_inativo_encontrado_outros['ibge_residencia_antigo'] = caso_inativo_encontrado_outros['ibge_residencia']
caso_inativo_encontrado_outros.loc[caso_inativo_encontrado_outros['ibge_residencia'] != 410690, 'ibge_residencia'] = 410690
caso_inativo_encontrado_outros['hash_resid'] = caso_inativo_encontrado_outros['paciente'].apply(normalize_hash) + caso_inativo_encontrado_outros['ibge_residencia'].astype(str)
caso_inativo_encontrado_outros['hash_resid_less'] = caso_inativo_encontrado_outros['paciente'].apply(normalize_hash) + caso_inativo_encontrado_outros['idade'].apply(less) + caso_inativo_encontrado_outros['ibge_residencia'].astype(str)
caso_inativo_encontrado_outros['hash_resid_more'] = caso_inativo_encontrado_outros['paciente'].apply(normalize_hash) + caso_inativo_encontrado_outros['idade'].apply(more) + caso_inativo_encontrado_outros['ibge_residencia'].astype(str)

In [ ]:
canditados_inativos_alteracao_de_municipio = caso_inativo_encontrado_outros.loc[~(
    (caso_inativo_encontrado_outros['hash_resid'].isin(cc.df['hash_resid']))
)]
print('*CANDITADOS INATIVOS A MUDANÇA DE MUNICIPIO PARA CURITIBA*')
canditados_inativos_alteracao_de_municipio.shape[0]

In [ ]:
canditados_inativos_alteracao_de_municipio = canditados_inativos_alteracao_de_municipio.merge(how = 'left', on = 'ibge_residencia', right=mn)
canditados_inativos_alteracao_de_municipio = canditados_inativos_alteracao_de_municipio.merge(how = 'left', on = 'ibge_residencia_antigo', right=mn[['ibge_residencia', 'municipio']].rename(columns={'ibge_residencia':'ibge_residencia_antigo', 'municipio':'municipio_antigo'}))
canditados_inativos_alteracao_de_municipio.loc[canditados_inativos_alteracao_de_municipio['evolucao']==2, 'obito'] = 'SIM'
#canditados_inativos_alteracao_de_municipio[['municipio_antigo','municipio','obito', 'sexo', 'idade']].to_csv(join(default_output,'canditados_inativos_alteracao_de_municipio_relatorio.csv'), encoding='utf-8-sig', sep=';')
#canditados_inativos_alteracao_de_municipio[['identificacao','ibge7','municipio_normalizado']].rename(columns={'identificacao':'Identificacao', 'ibge7':'ibge', 'municipio_normalizado':'mun resid'}).to_csv(join(default_output,'canditados_inativos_alteracao_de_municipio.csv'), encoding='utf-8-sig', sep=';')

In [ ]:
caso_inativo_nao_encontrado = caso_inativo.loc[~(
    (caso_inativo['id'].notna() & caso_inativo['id'].isin(cc.df['id_notifica'])) |
    (caso_inativo['hash_mae'].notna() & caso_inativo['hash_mae'].isin(cc.df['hash_mae'])) |
    (caso_inativo['hash_nasc'].notna() & caso_inativo['hash_nasc'].isin(cc.df['hash_nasc'])) 
)].copy()
print('VERIFICAR SE DA PRA ENTRAR*')
# print('Casos ativos não encontrado no pickle:')
# caso_inativo_nao_encontrado.to_csv(join(database_dir,'caso_inativo_nao_encontrado.csv'), encoding='utf-8-sig', sep=';', index=False)
# caso_inativo_nao_encontrado.to_pickle(join(database_dir,'caso_inativo_nao_encontrado.pkl'),'bz2')
caso_inativo_nao_encontrado.shape[0]

In [ ]:
print('*RESUMO DA ÓPERA*')
print(f"DOS {caso_curitiba.shape[0]} CASOS NOTIFICADOS PELA QUALIFICAÇÃO DE CURITIBA")
print(f"Casos notificados ativos que entraram no dia 26: {curitiba_entraram_ativo.shape[0]}")
print(f"Casos notificados ativos que já qualificamos: {caso_ativo.shape[0] - caso_ativo_nao_encontrado.shape[0] - curitiba_entraram_ativo.shape[0]}")
print(f"Destes, quantos não residentes em Curitiba (*POSSIVEL ALTERACAO DE MUNICIPIO*): {canditados_ativos_alteracao_de_municipio.shape[0]}")
print(f"Casos notificados inativos que entraram no dia 26: {curitiba_entraram_inativo.shape[0]}")
print(f"Casos notificados inativos que já qualificamos: {caso_inativo.shape[0] - caso_inativo_nao_encontrado.shape[0] - curitiba_entraram_inativo.shape[0]}")
print(f"Destes, quantos não residentes em Curitiba (*POSSIVEL ALTERACAO DE MUNICIPIO*): {canditados_inativos_alteracao_de_municipio.shape[0]}")
print(f"Casos ativos que não divulgamos (*VERIFICAR PORQUE NAO ENTROU*): {caso_ativo_nao_encontrado.shape[0]}")
print(f"Casos inativos que não divulgamos (*VERIFICAR SE DA PRA ENTRAR*): {caso_inativo_nao_encontrado.shape[0]}")

In [ ]:
caso_ativo_nao_encontrado

In [ ]:
caso_inativo_nao_encontrado

In [ ]:
raise
print('*VERIFICAÇÃO DO TOTAL*')
caso_ativo.shape[0] - caso_ativo_nao_encontrado.shape[0] + caso_inativo.shape[0] - caso_inativo_nao_encontrado.shape[0] + caso_ativo_nao_encontrado.shape[0] + caso_inativo_nao_encontrado.shape[0]

In [ ]:
# caso_ativo.loc[(
#     (caso_ativo['id'].isin(cc.df['id_notifica'])) |
#     (caso_ativo['hash_mae'].isin(cc.df['hash_mae'])) |
#     (caso_ativo['hash_nasc'].isin(cc.df['hash_nasc'])) |
#     (caso_ativo.loc[~caso_ativo['cns'].isna()].isin(cc.df.loc[~cc.df['cns'].isna()])) |
#     (caso_ativo.loc[~caso_ativo['cpf'].isna()].isin(cc.df.loc[~cc.df['cpf'].isna()])) |
#     (caso_ativo['hash_resid'].isin(cc.df['hash_resid'])) |
#     (caso_ativo['hash_resid_less'].isin(cc.df['hash_resid_less'])) |
#     (caso_ativo['hash_resid_more'].isin(cc.df['hash_resid_more'])) |
#     (caso_ativo['hash_atend'].isin(cc.df['hash_atend'])) |
#     (caso_ativo['hash_atend_less'].isin(cc.df['hash_atend_less'])) |
#     (caso_ativo['hash_atend_more'].isin(cc.df['hash_atend_more'])) |
#     (caso_ativo['hash_diag'].isin(cc.df['hash_diag']))
# )].shape[0]

In [ ]:
# caso_inativo.loc[(
#     (caso_inativo['id'].isin(cc.df['id_notifica'])) |
#     (caso_inativo['hash_mae'].isin(cc.df['hash_mae'])) |
#     (caso_inativo['hash_nasc'].isin(cc.df['hash_nasc'])) |
#     (caso_inativo.loc[~caso_ativo['cns'].isna()].isin(cc.df.loc[~cc.df['cns'].isna()])) |
#     (caso_inativo.loc[~caso_ativo['cpf'].isna()].isin(cc.df.loc[~cc.df['cpf'].isna()])) |
#     (caso_inativo['hash_resid'].isin(cc.df['hash_resid'])) |
#     (caso_inativo['hash_resid_less'].isin(cc.df['hash_resid_less'])) |
#     (caso_inativo['hash_resid_more'].isin(cc.df['hash_resid_more'])) |
#     (caso_inativo['hash_atend'].isin(cc.df['hash_atend'])) |
#     (caso_inativo['hash_atend_less'].isin(cc.df['hash_atend_less'])) |
#     (caso_inativo['hash_atend_more'].isin(cc.df['hash_atend_more'])) |
#     (caso_inativo['hash_diag'].isin(cc.df['hash_diag']))
# )].shape[0]

In [ ]:
del cc

In [ ]:
check_duplicates(caso_ativo_nao_encontrado)
caso_ativo_nao_encontrado.loc[caso_ativo_nao_encontrado['duplicated']].shape[0]

In [ ]:
print(f"{str(int(caso_ativo_nao_encontrado.loc[caso_ativo_nao_encontrado['duplicated']].shape[0]/caso_ativo_nao_encontrado.shape[0]*100))}% das fichas ativas que não entraram ({caso_ativo_nao_encontrado.shape[0]}) são fichas duplicadas")

In [ ]:
check_duplicates(caso_inativo_nao_encontrado)
caso_inativo_nao_encontrado.loc[caso_inativo_nao_encontrado['duplicated']].shape[0]

In [ ]:
print(f"{str(int(caso_inativo_nao_encontrado.loc[caso_inativo_nao_encontrado['duplicated']].shape[0]/caso_inativo_nao_encontrado.shape[0]*100))}% das fichas inativas que não entraram ({caso_inativo_nao_encontrado.shape[0]}) são fichas duplicadas")

In [ ]:
canditados_alteracao_de_municipio = pd.concat([canditados_ativos_alteracao_de_municipio, canditados_inativos_alteracao_de_municipio]).sort_values('id_notifica')
check_duplicates(canditados_alteracao_de_municipio)

In [ ]:
canditados_alteracao_de_municipio = canditados_alteracao_de_municipio.loc[(~(canditados_alteracao_de_municipio['duplicated'])) & (~(canditados_alteracao_de_municipio['obito']=='SIM'))]
canditados_alteracao_de_municipio.shape[0]

In [ ]:
#canditados_alteracao_de_municipio.loc[((canditados_alteracao_de_municipio['duplicated'])) & ((canditados_alteracao_de_municipio['obito']=='SIM'))].shape[0]

In [ ]:
canditados_alteracao_de_municipio[['municipio_antigo','municipio','obito', 'sexo', 'idade']].to_csv(join(default_output,'alteracao_de_municipio_relatorio.csv'), encoding='utf-8-sig', sep=';', index=False)
canditados_alteracao_de_municipio[['identificacao','ibge7','municipio_normalizado']].rename(columns={'identificacao':'Identificacao', 'ibge7':'ibge', 'municipio_normalizado':'mun resid'}).to_excel(join(default_output,'alteracao_de_municipio.xlsx'), encoding='utf-8-sig', index=False)